In [1]:
from glob import glob
import shutil
import argparse
import zipfile
import hashlib
import requests
from tqdm import tqdm
import IPython.display as display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from IPython.display import clear_output
import os
import hashlib
import torch
import torch.nn as nn
import copy
from torch.utils.data import Dataset, DataLoader
from torch import nn
from transformers import AutoImageProcessor
from PIL import Image
import torchvision.transforms as transforms
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix

In [2]:
def save_checkpoint(model, optimizer, loss, epoch, checkpoint_path="model_checkpoint.pth"):
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'loss': loss,
    }, checkpoint_path)
    print(f"Checkpoint saved at epoch {epoch+1}")

# Load checkpoint function
def load_checkpoint(model, optimizer, checkpoint_path="model_checkpoint.pth"):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    loss = checkpoint['loss']
    print(f"Checkpoint loaded. Resuming from epoch {start_epoch} with loss {loss:.4f}.")
    return start_epoch

In [ ]:
# Check SHA1 hash to ensure file integrity
def check_sha1(filename, sha1_hash):
    sha1 = hashlib.sha1()
    with open(filename, 'rb') as f:
        while True:
            data = f.read(1048576)
            if not data:
                break
            sha1.update(data)
    return sha1.hexdigest()[:len(sha1_hash)] == sha1_hash

# Download the dataset from the URL
def download(url, path=None, overwrite=False, sha1_hash=None):
    if path is None:
        fname = url.split('/')[-1]
    else:
        fname = os.path.join(path, url.split('/')[-1]) if os.path.isdir(path) else path

    if overwrite or not os.path.exists(fname) or (sha1_hash and not check_sha1(fname, sha1_hash)):
        os.makedirs(os.path.dirname(fname), exist_ok=True)
        print(f'Downloading {fname} from {url}...')
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(fname, 'wb') as f:
                for chunk in tqdm(r.iter_content(chunk_size=1024), unit='KB', dynamic_ncols=True):
                    f.write(chunk)
        if sha1_hash and not check_sha1(fname, sha1_hash):
            raise ValueError(f'File {fname} does not match hash {sha1_hash}')
    return fname

# Download ADE20K dataset
def download_ade(path, overwrite=False):
    if not os.path.exists(path):
        os.mkdir(path)
    _AUG_DOWNLOAD_URLS = [
        ('http://data.csail.mit.edu/places/ADEchallenge/ADEChallengeData2016.zip', '219e1696abb36c8ba3a3afe7fb2f4b4606a897c7'),
    ]
    download_dir = os.path.join(path, 'downloads')
    os.makedirs(download_dir, exist_ok=True)
    for url, checksum in _AUG_DOWNLOAD_URLS:
        filename = download(url, path=download_dir, overwrite=overwrite, sha1_hash=checksum)
        with zipfile.ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall(path=path)

# Dataset paths
root = "./"
dataset_path = os.path.join(root, "ADEChallengeData2016/images/")
training_data = "training/"
val_data = "validation/"

download_ade(root, overwrite=False)

N_CLASSES = 151
IMG_SIZE = 224

# Load image and mask paths
train_images = sorted(glob(os.path.join(dataset_path, training_data, "*.jpg")))
train_masks = sorted(glob(os.path.join(root, "ADEChallengeData2016/annotations/training/*.png")))
val_images = sorted(glob(os.path.join(dataset_path, val_data, "*.jpg")))
val_masks = sorted(glob(os.path.join(root, "ADEChallengeData2016/annotations/validation/*.png")))


In [ ]:
# Dataset class
class ADE20KDataset(Dataset):
    def __init__(self, image_paths, mask_paths, img_size=224):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.image_transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        self.mask_transform = transforms.Compose([
            transforms.Resize((img_size, img_size), interpolation=Image.NEAREST),
            transforms.Lambda(lambda x: torch.from_numpy(np.array(x, dtype=np.int64))),
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        mask = Image.open(self.mask_paths[idx]).convert("P")
        image = self.image_transform(image)
        mask = self.mask_transform(mask).squeeze(0).long()
        return image, mask

# Segmentation model
class DINOv2Segmentation(torch.nn.Module):
    def __init__(self, base_model, num_classes, embed_dim=384, freeze_backbone=True):
        super(DINOv2Segmentation, self).__init__()
        self.base_model = base_model
        self.decoder = torch.nn.Sequential(
            torch.nn.Conv2d(embed_dim, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 128, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(128, num_classes, kernel_size=1),
        )

        # Freeze backbone if specified
        if freeze_backbone:
            for param in self.base_model.parameters():
                param.requires_grad = False

    def forward(self, x):
        features = self.base_model.get_intermediate_layers(x, n=1)[0]
        B, N, C = features.size()  # Batch size, Tokens, Channels
        H = W = int(N ** 0.5)  # Square spatial resolution
        features = features.permute(0, 2, 1).view(B, C, H, W)  # Reshape to (B, C, H, W)
        return self.decoder(features)

# Training setup
train_dataset = ADE20KDataset(train_images, train_masks, img_size=IMG_SIZE)
val_dataset = ADE20KDataset(val_images, val_masks, img_size=IMG_SIZE)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').to(device)

# Initialize the model with frozen backbone
model = DINOv2Segmentation(dinov2_vits14, N_CLASSES, embed_dim=384, freeze_backbone=True).to(device)

# Only include trainable parameters in the optimizer
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()
epochs = 150

checkpoint_path = "frozen_model_checkpoint.pth"

# Attempt to load checkpoint
resume_training = True
if resume_training:
    try:
        start_epoch = load_checkpoint(model, optimizer, checkpoint_path)
    except FileNotFoundError:
        print("No checkpoint found. Starting training from scratch.")
        start_epoch = 0
else:
    start_epoch = 1

print(f"Starting training from epoch {start_epoch}.")
# Training loop with tqdm progress bar
for epoch in range(start_epoch, epochs):
    model.train()
    with tqdm(train_loader, unit='batch', desc=f'Epoch {epoch+1}/{epochs}') as tepoch:
        for images, masks in tepoch:
            images, masks = images.to(device), masks.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)  # Shape: [B, N_CLASSES, H, W]

            # Upsample the outputs to match the size of the masks
            outputs_upsampled = F.interpolate(outputs, size=masks.shape[1:], mode='bilinear', align_corners=False)

            # Compute the loss
            loss = criterion(outputs_upsampled, masks)
            loss.backward()
            optimizer.step()

            # Update tqdm description with loss information
            tepoch.set_postfix(loss=loss.item())

    save_checkpoint(model, optimizer, loss.item(), epoch, checkpoint_path)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")


"""
# Training loop with limited batches
for epoch in range(epochs):
    model.train()
    max_batches = 100  # Number of batches to test
    batch_count = 0
    with tqdm(train_loader, unit='batch', desc=f'Epoch {epoch+1}/10') as tepoch:
        for images, masks in tepoch:
            if batch_count >= max_batches:  # Stop after max_batches
                break

            images, masks = images.to(device), masks.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)  # Shape: [B, N_CLASSES, H, W]

            # Upsample the outputs to match the size of the masks
            outputs_upsampled = F.interpolate(outputs, size=masks.shape[1:], mode='bilinear', align_corners=False)

            # Compute the loss
            loss = criterion(outputs_upsampled, masks)
            loss.backward()
            optimizer.step()

            # Update tqdm description with loss information
            tepoch.set_postfix(loss=loss.item())

            batch_count += 1  # Increment batch counter

    print(f"Epoch {epoch+1}/10, Loss: {loss.item()}")
"""

In [ ]:
# Visualization function
def visualize_predictions(model, loader, device):
    model.eval()
    images, masks = next(iter(loader))
    images, masks = images.to(device), masks.to(device)

    with torch.no_grad():
        outputs = model(images)
        outputs_upsampled = F.interpolate(outputs, size=masks.shape[1:], mode="bilinear", align_corners=False)
        predictions = torch.argmax(outputs_upsampled, dim=1).cpu().numpy()

    # Visualize first batch
    for i in range(min(4, images.size(0))):
        img = images[i].cpu().permute(1, 2, 0).numpy()
        img = (img * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]  # De-normalize
        img = np.clip(img, 0, 1)

        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.imshow(img)
        plt.title("Input Image")

        plt.subplot(1, 3, 2)
        plt.imshow(masks[i].cpu(), cmap="viridis")
        plt.title("Ground Truth Mask")

        plt.subplot(1, 3, 3)
        plt.imshow(predictions[i], cmap="viridis")
        plt.title("Predicted Mask")
        plt.show()

# Visualize some predictions
visualize_predictions(model, val_loader, device)

In [ ]:
# Function to calculate IoU for each class
def compute_iou(pred, target, n_classes=N_CLASSES):
    iou_list = []
    for i in range(n_classes):
        # Create a binary mask for class i
        pred_i = (pred == i).float()
        target_i = (target == i).float()

        intersection = torch.sum(pred_i * target_i)
        union = torch.sum(pred_i) + torch.sum(target_i) - intersection

        iou = intersection / union if union != 0 else torch.tensor(0.0)
        iou_list.append(iou)
    return iou_list

# Function to display images and masks
def visualize_predictions(images, masks, preds, idx=0):
    # Image
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(images[idx].permute(1, 2, 0).cpu().numpy())
    axes[0].set_title("Image")

    # Ground Truth
    axes[1].imshow(masks[idx].cpu().numpy())
    axes[1].set_title("Ground Truth")

    # Prediction
    axes[2].imshow(preds[idx].cpu().numpy())
    axes[2].set_title("Prediction")

    plt.show()

# Testing loop
def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    all_iou = []  # To store IoU for each class across all batches

    # No gradient calculation needed during inference
    with torch.no_grad():
        # Loop through the test dataset
        for images, masks in tqdm(test_loader, desc="Testing"):
            images, masks = images.to(device), masks.to(device)

            # Forward pass: Get the outputs of the model
            outputs = model(images)  # Shape: [B, N_CLASSES, 16, 16]

            # Upsample the outputs to match the mask shape
            outputs_upsampled = F.interpolate(outputs, size=masks.shape[1:], mode='bilinear', align_corners=False)

            # Get the predictions (class with highest probability)
            preds = torch.argmax(outputs_upsampled, dim=1)  # Shape: [B, H, W]

            # Compute IoU for each batch
            iou_list = compute_iou(preds, masks)
            all_iou.append(iou_list)

            # Optionally visualize the predictions for the first image
            visualize_predictions(images, masks, preds, idx=0)

    # Calculate average IoU across all batches
    avg_iou = np.mean(all_iou, axis=0)
    print(f"Average IoU per class: {avg_iou}")

    return avg_iou

test_dataset = ADE20KDataset(val_images, val_masks, img_size=IMG_SIZE)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

# Run the test
avg_iou = test_model(model, test_loader)

In [5]:
def compute_miou(predictions, ground_truths, num_classes):
    """
    Computes the mean Intersection over Union (mIoU) with tqdm progress bar.

    Args:
        predictions (Tensor): Predicted segmentation maps, shape (B, H, W).
        ground_truths (Tensor): Ground truth segmentation maps, shape (B, H, W).
        num_classes (int): Number of classes.

    Returns:
        float: mIoU score.
    """
    ious = []
    for cls in tqdm(range(num_classes), desc="Calculating IoU for classes", unit="class"):
        intersection = ((predictions == cls) & (ground_truths == cls)).sum().item()
        union = ((predictions == cls) | (ground_truths == cls)).sum().item()
        if union == 0:
            ious.append(float('nan'))  # Ignore this class in mIoU computation
        else:
            ious.append(intersection / union)
    miou = np.nanmean(ious)  # Average over all valid classes
    return miou


In [9]:
def evaluate_model(model, loader, device, num_classes):
    model.eval()
    all_predictions = []
    all_ground_truths = []

    with torch.no_grad():
        for images, masks in tqdm(loader, desc="Evaluating"):
            images, masks = images.to(device), masks.to(device)

            # Forward pass
            outputs = model(images)
            outputs_upsampled = F.interpolate(outputs, size=masks.shape[1:], mode="bilinear", align_corners=False)
            predictions = torch.argmax(outputs_upsampled, dim=1)  # Shape: (B, H, W)

            all_predictions.append(predictions.cpu())
            all_ground_truths.append(masks.cpu())

    # Concatenate all batches
    all_predictions = torch.cat(all_predictions, dim=0)
    all_ground_truths = torch.cat(all_ground_truths, dim=0)

    # Compute mIoU
    miou = compute_miou(all_predictions, all_ground_truths, num_classes=num_classes)
    return miou

In [7]:
def evaluate_model_multiscale(model, loader, device, num_classes, scales=[0.5, 1.0, 1.5, 2.0]):
    model.eval()
    all_predictions = []
    all_ground_truths = []

    with torch.no_grad():
        for images, masks in tqdm(loader, desc="Evaluating (Multiscale)"):
            images, masks = images.to(device), masks.to(device)
            batch_size, _, orig_height, orig_width = images.size()

            # Multiscale predictions
            scale_predictions = []
            for scale in scales:
                scaled_images = F.interpolate(images, scale_factor=scale, mode='bilinear', align_corners=False)
                outputs = model(scaled_images)
                outputs_upsampled = F.interpolate(outputs, size=(orig_height, orig_width), mode="bilinear", align_corners=False)
                scale_predictions.append(outputs_upsampled)

            # Average predictions across scales
            averaged_outputs = torch.mean(torch.stack(scale_predictions, dim=0), dim=0)
            predictions = torch.argmax(averaged_outputs, dim=1)  # Shape: (B, H, W)

            all_predictions.append(predictions.cpu())
            all_ground_truths.append(masks.cpu())

    # Concatenate all batches
    all_predictions = torch.cat(all_predictions, dim=0)
    all_ground_truths = torch.cat(all_ground_truths, dim=0)

    # Compute mIoU
    miou = compute_miou(all_predictions, all_ground_truths, num_classes=num_classes)
    return miou


In [ ]:
# Linear evaluation (single scale)
test_dataset = ADE20KDataset(val_images, val_masks, img_size=IMG_SIZE)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

# Evaluate the model on the test dataset
miou_test = evaluate_model(model, test_loader, device, num_classes=N_CLASSES)
print(f"Mean IoU on ADE20K test set: {miou_test:.4f}")

# Multiscale evaluation on test set
miou_multiscale_test = evaluate_model_multiscale(model, test_loader, device, num_classes=N_CLASSES, scales=[0.5, 1.0, 1.5, 2.0])
print(f"Multiscale Mean IoU on ADE20K test set: {miou_multiscale_test:.4f}")